In [ ]:
# ===== KOBERT 기반 문장 임베딩 및 클러스터링 =====
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

# 문장 데이터
sentences =  [
    "가격 대비 성능이 정말 좋아요!",
    "이 정도 가격이면 훌륭한 선택 같아요.",
    "가성비 최고입니다. 만족스러워요.",
    "좀 비싸지만 성능이 괜찮네요.",
    "디자인이 너무 세련되고 예뻐요.",
    "가격이 너무 비싸서 고민되네요.",
    "이 정도 성능에 이 가격이면 괜찮은 듯.",
    "성능이 기대 이하여서 좀 아쉬워요.",
    "배터리 수명이 너무 짧아요...",
    "디자인이 깔끔하고 고급스러워 보이네요.",
    "가격이 좀만 더 저렴했으면 좋겠어요.",
    "성능이 빠릿빠릿하고 좋네요!",
    "디자인이 마음에 드는데 성능은 보통이네요.",
    "발열이 좀 심한 편이네요.",
    "이 정도면 가격 대비 충분한 성능이에요.",
    "너무 싸서 걱정했는데 의외로 괜찮아요.",
    "디자인이 심플하고 무난한 느낌이에요.",
    "화면 밝기가 너무 어두워서 별로네요.",
    "가격이 좀 높은데 성능은 확실히 좋아요.",
    "디자인이 투박해서 별로예요.",
    "가성비는 좋은데 내구성이 아쉬워요.",
    "소음이 심해서 사용하기 불편해요.",
    "성능이 뛰어나지만 가격이 부담되네요.",
    "이 가격에 이런 성능이라니 만족스럽네요.",
    "화면이 크고 선명해서 마음에 들어요.",
    "이전 모델보다 확실히 성능이 개선됐어요.",
    "고급스럽지만 가격이 너무 비싸요.",
    "디자인은 예쁘지만 실용성은 떨어지네요.",
    "소프트웨어 최적화가 좀 아쉬워요.",
    "가격이 내려가면 구매할 의향 있어요.",
    "디자인이 예쁜데 스크래치가 잘 생겨요.",
    "가격이 싸서 기대 안 했는데 성능이 괜찮아요.",
    "발열도 없고 성능이 상당히 좋아요.",
    "조작이 쉬워서 누구나 사용하기 편할 듯해요.",
    "성능이 좋긴 한데 배터리가 빨리 닳아요.",
    "이 가격이면 이 정도 성능은 기본 아닌가요?",
    "크기가 너무 커서 휴대성이 떨어지네요.",
    "소음이 적고 조용하게 작동하네요.",
    "디자인이 너무 독특해서 호불호 갈릴 듯해요.",
    "화면 주사율이 높아서 부드러워요!",
    "성능은 뛰어난데 무게가 좀 있어요.",
    "이 정도 성능이면 충분히 만족스러워요.",
    "가격이 더 저렴했으면 대박인데 아쉽네요.",
    "배터리가 오래가서 여행할 때 좋겠어요.",
    "조립이 좀 어려운 편이네요.",
    "디자인과 성능 모두 만족스럽습니다!",
    "가성비는 좋은데 브랜드가 좀 아쉬워요.",
    "디자인이 너무 투박해서 별로예요.",
    "가격 대비 성능이 부족한 느낌이에요.",
    "이전 모델보다 디자인이 확실히 예뻐졌네요."
]
# KOBERT 임베딩 준비
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertModel.from_pretrained("monologg/kobert")
model.eval()

# 문장을 BERT 임베딩 벡터로 변환 (CLS 토큰 사용)
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

# 전체 문장 임베딩
sentence_vectors = np.array([get_bert_embedding(sent) for sent in sentences])

# KMeans 클러스터링
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(sentence_vectors)
labels = kmeans.labels_

# PCA 차원 축소
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(sentence_vectors)

# 시각화
plt.figure(figsize=(10, 7))
scatter = plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], c=labels, cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label="Cluster")
plt.title("K-Means Clustering using KoBERT Embeddings (PCA Reduced)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")

# 문장 번호 추가
for i, txt in enumerate(sentences):
    plt.annotate(i, (reduced_vectors[i, 0], reduced_vectors[i, 1]), fontsize=8, alpha=0.7)

plt.show()

# 클러스터 결과 출력
for i, label in enumerate(labels):
    print(f"문장 {i}: {sentences[i]} → 클러스터 {label}")

